# ez-optimize Examples

This notebook provides examples for the `ez-optimize` library, demonstrating core and advanced features. For a quick introduction, see the [README.md](../README.md).

We use the Rosenbrock function due to its simplicity and popularity as an optimization benchmark.

## Table of Contents
1. [Simple Keyword-Based Optimization](#example-1)
2. [Maximization](#example-2)
3. [Dictionary 2D with Bounds](#example-3)
4. [Dictionary with Array Value](#example-4)
5. [Manual Control with OptimizationProblem.optimize()](#example-5)
6. [Full Manual Control](#example-6)
7. [Typical Optimizer Usage (5 Variables with Bounds)](#example-7)

In [1]:
# Setup: Import libraries and define example functions
import numpy as np
from functools import partial
from ez_optimize import minimize, OptimizationProblem

# 2D Rosenbrock function (known minimum: value=0 @ x=1, y=1, a=1, b=100)
def rosenbrock_2d(x, y, a, b):
    return (a - x)**2 + b * (y - x**2)**2

# N-Dimensional Rosenbrock for arrays
def rosen(x, a, b):
    return sum(a * (x[1:] - x[:-1] ** 2.0) ** 2.0 + (1 - x[:-1]) ** 2.0) + b

### <span id='example-1'>Example 1: Simple Keyword-Based Optimization</span>

**Goal**: Minimize the Rosenbrock function using keyword-based parameters.

Note: By default, optimization uses arrays `x0=[1, 2]`. However, it's more intuitive to use named parameters `x0={'x': 1, 'y': 2}`. `ez-optimize` allows you to define parameters as dictionaries. Under the hood, `ez-optimize` automatically flattens parameters (and wraps your function) for SciPy while restoring the original structure in results.

In [2]:
x0 = {'x': 1.3, 'y': 0.7}

result = minimize(partial(rosenbrock_2d, a=1, b=100), x0, method='BFGS')

print(f"Optimal x: {result.x}")
print(f"Optimal value: {result.fun}")
print(f"x_map: {result._x_map}")

Optimal x: {'x': array(0.99999552), 'y': array(0.99999102)}
Optimal value: 2.0115094474352237e-11
x_map: {'x': (), 'y': ()}


### <span id='example-2'>Example 2: Maximization</span>

**Goal**: Maximize a function using `direction='max'`.

Note: By default, optimization minimizes the objective function. To maximize, you typically need to write a negated version of your function. With `ez-optimize`, simply set `direction='max'` and the library will automatically negate your function under the hood.

In [3]:
def f_kw(x):
    return - (x - 1)**2

result = minimize(f_kw, {'x': 0.}, method='SLSQP', direction='max')

print(f"Optimal x: {result.x}")
print(f"Optimal value: {result.fun}")

Optimal x: {'x': array(1.)}
Optimal value: -4.930380657631324e-32


### <span id='example-3'>Example 3: Dictionary 2D with Bounds</span>

**Goal**: Optimize a 2D Rosenbrock function with keyword parameters and bounds.

In [4]:
x0 = {'x': 1.3, 'y': 0.7}
bounds = {'x': (0, 2), 'y': (0, 2)}

result = minimize(partial(rosenbrock_2d, a=1, b=100), x0, method='SLSQP', args=(1, 100), bounds=bounds)

print(f"Optimal x: {result.x}")
print(f"Optimal value: {result.fun}")

Optimal x: {'x': array(0.99991056), 'y': array(0.99981446)}
Optimal value: 1.2438732728332328e-08


### <span id='example-4'>Example 4: Dictionary with Array Value</span>

**Goal**: Optimize with a dictionary containing an array parameter.

In [5]:
x0 = {'x': np.array([1.3, 0.7, 0.8, 1.9, 1.2])}

result = minimize(partial(rosen, a=1, b=100), x0, method='BFGS', args=(100, 0))

print(f"Optimal x: {result.x}")
print(f"Optimal value: {result.fun}")

Optimal x: {'x': array([1.00000002, 0.99999994, 1.00000006, 1.00000015, 1.00000024])}
Optimal value: 100.00000000000009


### <span id='example-5'>Example 5: Manual Control with OptimizationProblem.optimize()</span>

**Goal**: Use the `OptimizationProblem` class for more control, calling `optimize()` method.

In [6]:
x0 = {'x': 1.3, 'y': 0.7}

prob = OptimizationProblem(partial(rosenbrock_2d, a=1, b=100), x0, method='BFGS', args=(1, 100))

result = prob.optimize()

print(f"Optimal x: {result.x}")
print(f"Optimal value: {result.fun}")

Optimal x: {'x': array(0.99999552), 'y': array(0.99999102)}
Optimal value: 2.0115094474352237e-11


### <span id='example-6'>Example 6: Full Manual Control</span>

**Goal**: For advanced manual control, use the `OptimizationProblem` class directly. This also serves as a look under the hood for how `minimize` works.

TODO: show how this can be used to slip in props to scipy or to implement a custom optimization loop

In [7]:
from scipy.optimize import minimize as scipy_minimize

def objective(a, b, c):
    return a**2 + b**2 + c**2

x0 = {'a': 1.0, 'b': 2.0, 'c': 3.0}
bounds = {'a': (0, 5), 'b': (0, 5), 'c': (0, 5)}

# Define the optimization problem
problem = OptimizationProblem(objective, x0, method='SLSQP', bounds=bounds)

# Run SciPy method directly, passing in the arguments prepared by the OptimizationProblem
scipy_result = scipy_minimize(**problem.scipy.get_minimize_args())

# Use the OptimizationProblem to interpret the result back into our structured format
result = problem.scipy.interpret_result(scipy_result)

print(f"Optimal parameters: {result.x}")
print(f"Optimal value: {result.fun}")

Optimal parameters: {'a': array(1.77635684e-15), 'b': array(4.4408921e-16), 'c': array(0.)}
Optimal value: 3.3526588471893e-30


### <span id='example-7'>Example 7: Typical Optimizer Usage (5 Variables with Bounds)</span>

**Goal**: Show that `ez-optimize` can still be used like a typical SciPy optimizer with arrays and bounds.

In [8]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
bounds = [(0, 2), (0, 2), (0, 2), (0, 2), (0, 2)]

result = minimize(rosen, x0, method='SLSQP', args=(100, 0), bounds=bounds)

print(f"Optimal x: {result.x}")
print(f"Optimal value: {result.fun}")

Optimal x: [1.00004303 1.0000684  1.00017009 1.0003454  1.0006957 ]
Optimal value: 3.0170901914560603e-07
